In [ ]:
# -------------------- 1. 导入库并加载必要文件 --------------------
import warnings
warnings.filterwarnings("ignore")

import geatpy as ea
import numpy as np
import pandas as pd
import joblib
import os
from tqdm import trange
from scipy.stats import entropy

# --- 配置区 (请根据您的环境修改路径) ---
MODEL_PATH = r'C:\Users\DELL\Desktop\wrq\lake_mp_optimization_project\model\random_forest_model.pkl'
CONSTRAINTS_PATH = r'C:\Users\DELL\Desktop\wrq\lake_mp_optimization_project\data\yueshu\final_lake_constraints.csv'
OUTPUT_DIR = r'C:\Users\DELL\Desktop\wrq' # 使用新的最终版输出目录

# --- 创建输出目录 (目录保持原名，文件名将包含标记) ---
OUTPUT_DIR_Y = os.path.join(OUTPUT_DIR, '最优目标值')
OUTPUT_DIR_X = os.path.join(OUTPUT_DIR, '最优解特征')
OUTPUT_DIR_ENTROPY = os.path.join(OUTPUT_DIR, '信息熵分析')
OUTPUT_DIR_CONTRIBUTION = os.path.join(OUTPUT_DIR, '贡献度分析')
os.makedirs(OUTPUT_DIR_Y, exist_ok=True)
os.makedirs(OUTPUT_DIR_X, exist_ok=True)
os.makedirs(OUTPUT_DIR_ENTROPY, exist_ok=True)
os.makedirs(OUTPUT_DIR_CONTRIBUTION, exist_ok=True)

# --- 加载模型和数据 (静默模式) ---
# print("正在加载模型和数据...")
model = joblib.load(MODEL_PATH)
data = pd.read_csv(CONSTRAINTS_PATH)
# print("加载完成！")

# --- 1.1 逐行读取所有60个特征 (优化为列表推导式以减少内存占用) ---
# print("正在逐一读取所有约束列...")
# 通过列表推导式直接生成列表的列表，避免创建60个中间变量
all_cols_list = [data.iloc[:, j].tolist() for j in range(60)]
# print("列数据读取完成！")

DIMENSION = 22 # 模型需要前22个特征

# -------------------- 2. 定义Geatpy优化问题类 --------------------

class MyProblemStrict(ea.Problem):
    def __init__(self, lb, ub, all_fixed_cols, row_index, model_predictor):
        name = 'StrictIndexProblem'; M = 1; maxormins = [1]; varTypes = [0] * DIMENSION
        ea.Problem.__init__(self, name, M, maxormins, DIMENSION, varTypes, lb, ub)
        self.all_cols = all_fixed_cols; self.row_index = row_index; self.model = model_predictor

    def aimFunc(self, pop):
        Vars = pop.Phen; i = self.row_index
        safe_vars = np.nan_to_num(Vars, nan=0.0)
        pop.ObjV = self.model.predict(safe_vars).reshape(-1, 1)
        epsilon = 1e-6
        ww_init_sum = np.nansum([self.all_cols[6][i], self.all_cols[7][i], self.all_cols[8][i]])
        rse_init_sum = np.nansum([self.all_cols[9][i], self.all_cols[10][i], self.all_cols[11][i]])
        ww_sec_adv_upper = np.nan_to_num(self.all_cols[50][i], nan=ww_init_sum)
        rse_paved_gravel_upper = np.nan_to_num(self.all_cols[53][i], nan=rse_init_sum)
        cv1 = np.abs((Vars[:, 6] + Vars[:, 7] + Vars[:, 8]) - ww_init_sum) - epsilon
        cv2 = np.abs((Vars[:, 9] + Vars[:, 10] + Vars[:, 11]) - rse_init_sum) - epsilon
        cv3 = (Vars[:, 7] + Vars[:, 8]) - ww_sec_adv_upper
        cv4 = (Vars[:, 9] + Vars[:, 10]) - rse_paved_gravel_upper
        pop.CV = np.hstack([cv1.reshape(-1, 1), cv2.reshape(-1, 1), cv3.reshape(-1, 1), cv4.reshape(-1, 1)])

# -------------------- 3. 主循环与优化过程 --------------------
results_Y, results_X, results_entropy, results_contribution = [], [], [], []
row_ids = data.index.tolist()

# --- 修改点：设定处理行数和起始行 ---
# Set the starting row index for this run
START_ROW = 15000 # Start from the 15,001st row (index 15000)
# Set the number of rows to process in this run
NUM_ROWS_TO_PROCESS = 15000
# Calculate the ending row index, ensuring it doesn't exceed total data length
END_ROW = min(START_ROW + NUM_ROWS_TO_PROCESS, len(data))

# Loop from START_ROW up to (but not including) END_ROW
for i in trange(START_ROW, END_ROW, desc=f"最终版逐行优化(行 {START_ROW} 到 {END_ROW-1})"):

    # 防御层 1: 检查核心特征，如果含NaN或inf则跳过整行
    model_input_features = [all_cols_list[j][i] for j in range(DIMENSION)]
    if any(pd.isna(v) or np.isinf(v) for v in model_input_features):
        results_Y.append(np.nan); results_X.append([np.nan] * DIMENSION)
        results_entropy.append([np.nan] * DIMENSION); results_contribution.append([np.nan] * DIMENSION)
        continue

    lb = np.zeros(DIMENSION); ub = np.zeros(DIMENSION)

    # 防御层 2: 逐个变量设置边界，如果约束依赖含NaN或inf则固定变量
    # 1. 固定变量
    for col_idx in [0, 1, 2, 3, 4, 5, 12, 13, 14, 15, 16, 18]:
        val = all_cols_list[col_idx][i]
        lb[col_idx] = val
        ub[col_idx] = val

    # 2. 可优化变量
    # 定义一个辅助函数来检查依赖项
    def check_deps(dep_list):
        return any(pd.isna(v) or np.isinf(v) for v in dep_list)

    # Primary_Waste_Discharge (idx 6)
    deps_6 = [all_cols_list[6][i], all_cols_list[7][i], all_cols_list[8][i], all_cols_list[51][i]]
    if check_deps(deps_6):
        lb[6] = ub[6] = all_cols_list[6][i]
    else:
        lb[6] = max(0, all_cols_list[51][i]); ub[6] = sum(deps_6[0:3])

    # Secondary_Waste_Discharge (idx 7)
    deps_7 = [all_cols_list[6][i], all_cols_list[7][i], all_cols_list[8][i]]
    if check_deps(deps_7):
        lb[7] = ub[7] = all_cols_list[7][i]
    else:
        lb[7] = max(0, all_cols_list[7][i]); ub[7] = sum(deps_7)

    # Advanced_Waste_Discharge (idx 8)
    deps_8 = [all_cols_list[8][i], all_cols_list[49][i]]
    if check_deps(deps_8):
        lb[8] = ub[8] = all_cols_list[8][i]
    else:
        lb[8] = max(0, all_cols_list[8][i]); ub[8] = all_cols_list[49][i]

    # RSE_paved (idx 9)
    deps_9 = [all_cols_list[9][i], all_cols_list[52][i]]
    if check_deps(deps_9):
        lb[9] = ub[9] = all_cols_list[9][i]
    else:
        lb[9] = max(0, all_cols_list[9][i]); ub[9] = all_cols_list[52][i]

    # RSE_gravel (idx 10)
    deps_10 = [all_cols_list[9][i], all_cols_list[10][i], all_cols_list[11][i]]
    if check_deps(deps_10):
        lb[10] = ub[10] = all_cols_list[10][i]
    else:
        lb[10] = max(0, all_cols_list[10][i]); ub[10] = sum(deps_10)

    # RSE_other (idx 11)
    deps_11 = [all_cols_list[9][i], all_cols_list[10][i], all_cols_list[11][i], all_cols_list[54][i]]
    if check_deps(deps_11):
        lb[11] = ub[11] = all_cols_list[11][i]
    else:
        lb[11] = max(0, all_cols_list[54][i]); ub[11] = sum(deps_11[0:3])

    # Mismanaged (idx 17)
    deps_17 = [all_cols_list[17][i], all_cols_list[55][i]]
    if check_deps(deps_17) or all_cols_list[17][i] == 0:
        lb[17] = ub[17] = all_cols_list[17][i]
    else:
        lb[17] = max(0, all_cols_list[55][i]); ub[17] = all_cols_list[17][i]

    # fish_gdp_sqkm (idx 19)
    deps_19 = [all_cols_list[19][i], all_cols_list[56][i], all_cols_list[57][i]]
    if check_deps(deps_19):
        lb[19] = ub[19] = all_cols_list[19][i]
    else:
        lb[19] = max(0, all_cols_list[19][i] * all_cols_list[56][i])
        ub[19] = all_cols_list[19][i] * all_cols_list[57][i]

    # Cultivated_land (idx 20)
    deps_20 = [all_cols_list[20][i], all_cols_list[31][i], all_cols_list[32][i]]
    if check_deps(deps_20):
        lb[20] = ub[20] = all_cols_list[20][i]
    else:
        lb[20] = max(0, all_cols_list[31][i]); ub[20] = all_cols_list[32][i]

    # Artificial_surface (idx 21)
    deps_21 = [all_cols_list[21][i], all_cols_list[33][i], all_cols_list[34][i]]
    if check_deps(deps_21):
        lb[21] = ub[21] = all_cols_list[21][i]
    else:
        lb[21] = max(0, all_cols_list[33][i]); ub[21] = all_cols_list[34][i]

    # 最终强制校正，以防浮点数精度导致 lb > ub
    fix_indices = np.where(lb > ub)[0]
    if len(fix_indices) > 0:
        lb[fix_indices] = ub[fix_indices]

    if np.any(lb > ub) or np.any(np.isnan(lb)) or np.any(np.isnan(ub)) or np.any(np.isinf(lb)) or np.any(np.isinf(ub)):
        results_Y.append(np.nan); results_X.append([np.nan] * DIMENSION)
        results_entropy.append([np.nan] * DIMENSION); results_contribution.append([np.nan] * DIMENSION)
        continue

    problem = MyProblemStrict(lb=lb, ub=ub, all_fixed_cols=all_cols_list, row_index=i, model_predictor=model)
    algorithm = ea.soea_EGA_templet(problem, ea.Population(Encoding='RI', NIND=50), MAXGEN=100, logTras=0)
    res = ea.optimize(algorithm, seed=1, verbose=False, drawing=0, outputMsg=False)

    if res['success']:
        best_y = res['optPop'].ObjV[0, 0]; best_x = res['optPop'].Phen[0, :]
        final_pop_vars = res['optPop'].Phen
        results_Y.append(best_y); results_X.append(best_x)
        entropies = [entropy(np.histogram(final_pop_vars[:, j], bins=10)[0] / final_pop_vars.shape[0]) for j in range(DIMENSION)]
        results_entropy.append(entropies)
        results_contribution.append(model.feature_importances_)
    else:
        results_Y.append(np.nan); results_X.append([np.nan] * DIMENSION)
        results_entropy.append([np.nan] * DIMENSION); results_contribution.append([np.nan] * DIMENSION)


# -------------------- 4. 保存结果到文件 --------------------
# print("\n优化完成，正在保存结果...")

# 定义特征名称以便于输出文件有表头
feature_names = [f'feature_{j}' for j in range(DIMENSION)]
# Modified file suffix to reflect the processed range
file_suffix = f"_{START_ROW}_to_{END_ROW-1}"

# 保存最优目标值Y
df_Y = pd.DataFrame(results_Y, columns=['optimal_target_value'])
df_Y.to_csv(os.path.join(OUTPUT_DIR_Y, f'optimal_Y{file_suffix}.csv'), index=False)

# 保存最优解X
df_X = pd.DataFrame(results_X, columns=feature_names)
df_X.to_csv(os.path.join(OUTPUT_DIR_X, f'optimal_X{file_suffix}.csv'), index=False)

# 保存信息熵分析
df_entropy = pd.DataFrame(results_entropy, columns=[f'entropy_{j}' for j in range(DIMENSION)])
df_entropy.to_csv(os.path.join(OUTPUT_DIR_ENTROPY, f'entropy_analysis{file_suffix}.csv'), index=False)

# 保存贡献度分析
df_contribution = pd.DataFrame(results_contribution, columns=[f'contribution_{j}' for j in range(DIMENSION)])
df_contribution.to_csv(os.path.join(OUTPUT_DIR_CONTRIBUTION, f'contribution_analysis{file_suffix}.csv'), index=False)

# print(f"所有结果已成功保存到目录: {OUTPUT_DIR}")